In [1]:
### Initial design of drawing a coplanar waveguide resonator 


# automatic reloading of modules when they change

%load_ext autoreload
%autoreload 2


In [2]:
# Import statements

import qiskit_metal as metal
from qiskit_metal import designs, draw
from qiskit_metal import MetalGUI, Dict, Headings
import pyEPR as epr

from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.terminations.open_to_ground import OpenToGround
from qiskit_metal.qlibrary.terminations.short_to_ground import ShortToGround
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight

In [3]:
# Instantiate QDesign class and initialise user interface

design = designs.DesignPlanar({}, True)
design.chips.main.size['size_x'] = '2mm'
design.chips.main.size['size_y'] = '2mm'
design.chips.main.size['size_z'] = '-500um'
design.chips.main.size['sample_holder_bottom'] = '500um'
design.chips.main.size['sample_holder_top'] = '2000um'


# Set hfss as renderer for design
hfss = design.renderers.hfss

In [4]:
# Allow overwriting
gui = MetalGUI(design)
design.overwrite_enabled = True

In [5]:
# Create shorted pin and open pin for resonator

otg1 = OpenToGround(design, 'open_to_ground1', options=dict(pos_x='0um', pos_y='-1.25mm',
orientation='180'))
stg = ShortToGround(design, 'short_to_ground', options=dict(pos_x='0um', pos_y='0.5mm',
orientation='270')) 


# Jogs to allow bending of open end of CPW
# So far does not render in hfss
from collections import OrderedDict
jogs = OrderedDict()
jogs[0] = ["L", '125um']
jogs[1] = ["R", '100um']
#jogs[2] = ["L", '100um']

jogs2 = OrderedDict()
jogs2[0] = ["R", '100um']
jogs2[1] = ["L", '110um' ]

# Create meandered resonator between shorted and opened pin
rt_meander = RouteMeander(design, 'readout', Dict(
        total_length='4.12 mm',
        hfss_wire_bonds = False,
        fillet='80 um',
        lead = Dict(
        start_straight='0.1mm',
        end_straight='0mm'),
        #start_jogged_extensions=jogs2,
        #end_jogged_extension=jogs),
        meander=Dict(
        spacing = '200um'),
        pin_inputs=Dict(
            start_pin=Dict(component='short_to_ground', pin='short'),
            end_pin=Dict(component='open_to_ground1', pin='open')), ))

gui.rebuild()
gui.autoscale()

In [6]:
hfss.open_ansys(path = 'C:\Program Files\AnsysEM\AnsysEM19.3\Win64')

In [7]:
hfss.new_ansys_project()

In [8]:
hfss.connect_ansys()

INFO 03:04PM [connect_project]: Connecting to Ansys Desktop API...
INFO 03:04PM [load_ansys_project]: 	Opened Ansys App
INFO 03:04PM [load_ansys_project]: 	Opened Ansys Desktop v2019.1.0
INFO 03:04PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/2254729M/Documents/Ansoft/
	Project:   Project7
INFO 03:04PM [connect_design]: No active design found (or error getting active design).
INFO 03:04PM [connect]: 	 Connected to project "Project7". No design detected


In [ ]:
hfss.clean_active_design()

In [9]:
hfss.activate_ansys_design("Readout", 'eigenmode')

03:04PM 56s WARNING [activate_ansys_design]: The design_name=Readout was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 03:04PM [connect_design]: 	Opened active design
	Design:    Readout [Solution type: Eigenmode]
WARNING 03:04PM [connect_setup]: 	No design setup detected.
WARNING 03:04PM [connect_setup]: 	Creating eigenmode default setup.
INFO 03:04PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [10]:
hfss.render_design(['short_to_ground', 'readout', 'open_to_ground1'],[])
#hfss.render_design(['short_to_ground', 'readout', 'open_to_ground'], [])

03:05PM 06s ERROR [render_element_path]: We cannot find a writable design. 
  Either you are trying to use a Ansys design that is not empty, in which case please clear it manually or with the renderer method clean_active_design(). 
  Or you accidentally deleted the design in Ansys, in which case please create a new one.


com_error:  (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024381), None)


com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2147024381), None)

In [ ]:
setup = hfss.pinfo.setup
setup.passes = 20
print(f"""
Number of eigenmodes to find             = {setup.n_modes}
Number of simulation passes              = {setup.passes}
Convergence freq max delta percent diff  = {setup.delta_f}
""")

pinfo = hfss.pinfo
pinfo.design.set_variable('Lj', '10 nH')
pinfo.design.set_variable('Cj', '0 fF')
setup.analyze()

hfss.plot_convergences()

In [ ]:
hfss.disconnect_ansys()